# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Oct 7 2022 8:30AM,248114,10,MSP214699,"Methapharm, Inc.",Released
1,Oct 6 2022 4:01PM,248113,15,30011143,"Alliance Pharma, Inc.",Released
2,Oct 6 2022 4:01PM,248113,15,30011144,"Alliance Pharma, Inc.",Executing
3,Oct 6 2022 4:01PM,248113,15,30011146,"Alliance Pharma, Inc.",Executing
4,Oct 6 2022 4:01PM,248113,15,30011147,"Alliance Pharma, Inc.",Released
5,Oct 6 2022 4:01PM,248113,15,30011149,"Alliance Pharma, Inc.",Executing
6,Oct 6 2022 4:01PM,248113,15,30011151,"Alliance Pharma, Inc.",Executing
7,Oct 6 2022 4:01PM,248113,15,30011148,"Alliance Pharma, Inc.",Released
8,Oct 6 2022 4:01PM,248113,15,30011150,"Alliance Pharma, Inc.",Released
9,Oct 6 2022 4:01PM,248113,15,30011152,"Alliance Pharma, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
11,248110,Released,1
12,248111,Released,1
13,248113,Executing,4
14,248113,Released,6
15,248114,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
248108,NaN,NaN,1.0
248110,NaN,NaN,1.0
248111,NaN,NaN,1.0
248113,NaN,4.0,6.0
248114,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
248000,6.0,13.0,1.0
248037,3.0,4.0,1.0
248071,0.0,0.0,1.0
248075,0.0,0.0,3.0
248085,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
248000,6,13,1
248037,3,4,1
248071,0,0,1
248075,0,0,3
248085,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,248000,6,13,1
1,248037,3,4,1
2,248071,0,0,1
3,248075,0,0,3
4,248085,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,248000,6,13,1
1,248037,3,4,1
2,248071,,,1
3,248075,,,3
4,248085,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Oct 7 2022 8:30AM,248114,10,"Methapharm, Inc."
1,Oct 6 2022 4:01PM,248113,15,"Alliance Pharma, Inc."
11,Oct 6 2022 4:01PM,248075,15,"Carwin Pharmaceutical Associates, LLC"
14,Oct 6 2022 3:42PM,248111,22,"NBTY Global, Inc."
15,Oct 6 2022 3:41PM,248110,22,"NBTY Global, Inc."
16,Oct 6 2022 3:25PM,248108,10,Eywa Pharma Inc.
17,Oct 6 2022 2:46PM,248097,16,Sartorius Bioprocess Solutions
18,Oct 6 2022 2:26PM,248085,20,"ACI Healthcare USA, Inc."
19,Oct 6 2022 1:37PM,248071,20,"ACI Healthcare USA, Inc."
20,Oct 6 2022 10:14AM,248037,15,"Person & Covey, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Oct 7 2022 8:30AM,248114,10,"Methapharm, Inc.",,,1
1,Oct 6 2022 4:01PM,248113,15,"Alliance Pharma, Inc.",,4,6
2,Oct 6 2022 4:01PM,248075,15,"Carwin Pharmaceutical Associates, LLC",,,3
3,Oct 6 2022 3:42PM,248111,22,"NBTY Global, Inc.",,,1
4,Oct 6 2022 3:41PM,248110,22,"NBTY Global, Inc.",,,1
5,Oct 6 2022 3:25PM,248108,10,Eywa Pharma Inc.,,,1
6,Oct 6 2022 2:46PM,248097,16,Sartorius Bioprocess Solutions,,1,
7,Oct 6 2022 2:26PM,248085,20,"ACI Healthcare USA, Inc.",,,1
8,Oct 6 2022 1:37PM,248071,20,"ACI Healthcare USA, Inc.",,,1
9,Oct 6 2022 10:14AM,248037,15,"Person & Covey, Inc.",3,4,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 7 2022 8:30AM,248114,10,"Methapharm, Inc.",1,,
1,Oct 6 2022 4:01PM,248113,15,"Alliance Pharma, Inc.",6,4,
2,Oct 6 2022 4:01PM,248075,15,"Carwin Pharmaceutical Associates, LLC",3,,
3,Oct 6 2022 3:42PM,248111,22,"NBTY Global, Inc.",1,,
4,Oct 6 2022 3:41PM,248110,22,"NBTY Global, Inc.",1,,
5,Oct 6 2022 3:25PM,248108,10,Eywa Pharma Inc.,1,,
6,Oct 6 2022 2:46PM,248097,16,Sartorius Bioprocess Solutions,,1,
7,Oct 6 2022 2:26PM,248085,20,"ACI Healthcare USA, Inc.",1,,
8,Oct 6 2022 1:37PM,248071,20,"ACI Healthcare USA, Inc.",1,,
9,Oct 6 2022 10:14AM,248037,15,"Person & Covey, Inc.",1,4,3


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 7 2022 8:30AM,248114,10,"Methapharm, Inc.",1,,
1,Oct 6 2022 4:01PM,248113,15,"Alliance Pharma, Inc.",6,4,
2,Oct 6 2022 4:01PM,248075,15,"Carwin Pharmaceutical Associates, LLC",3,,
3,Oct 6 2022 3:42PM,248111,22,"NBTY Global, Inc.",1,,
4,Oct 6 2022 3:41PM,248110,22,"NBTY Global, Inc.",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 7 2022 8:30AM,248114,10,"Methapharm, Inc.",1.0,NaN,NaN
1,Oct 6 2022 4:01PM,248113,15,"Alliance Pharma, Inc.",6.0,4.0,NaN
2,Oct 6 2022 4:01PM,248075,15,"Carwin Pharmaceutical Associates, LLC",3.0,NaN,NaN
3,Oct 6 2022 3:42PM,248111,22,"NBTY Global, Inc.",1.0,NaN,NaN
4,Oct 6 2022 3:41PM,248110,22,"NBTY Global, Inc.",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 7 2022 8:30AM,248114,10,"Methapharm, Inc.",1.0,0.0,0.0
1,Oct 6 2022 4:01PM,248113,15,"Alliance Pharma, Inc.",6.0,4.0,0.0
2,Oct 6 2022 4:01PM,248075,15,"Carwin Pharmaceutical Associates, LLC",3.0,0.0,0.0
3,Oct 6 2022 3:42PM,248111,22,"NBTY Global, Inc.",1.0,0.0,0.0
4,Oct 6 2022 3:41PM,248110,22,"NBTY Global, Inc.",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,496222,2.0,0.0,0.0
15,992225,11.0,21.0,9.0
16,248097,0.0,1.0,0.0
20,496156,2.0,0.0,0.0
22,496221,2.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,496222,2.0,0.0,0.0
1,15,992225,11.0,21.0,9.0
2,16,248097,0.0,1.0,0.0
3,20,496156,2.0,0.0,0.0
4,22,496221,2.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,2.0,0.0,0.0
1,15,11.0,21.0,9.0
2,16,0.0,1.0,0.0
3,20,2.0,0.0,0.0
4,22,2.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,2.0
1,15,Released,11.0
2,16,Released,0.0
3,20,Released,2.0
4,22,Released,2.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,20,22
Status,,,,,
Completed,0.0,9.0,0.0,0.0,0.0
Executing,0.0,21.0,1.0,0.0,0.0
Released,2.0,11.0,0.0,2.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,20,22
0,Completed,0.0,9.0,0.0,0.0,0.0
1,Executing,0.0,21.0,1.0,0.0,0.0
2,Released,2.0,11.0,0.0,2.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,20,22
0,Completed,0.0,9.0,0.0,0.0,0.0
1,Executing,0.0,21.0,1.0,0.0,0.0
2,Released,2.0,11.0,0.0,2.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()